Este projeto desenvolverá um sistema que utiliza agentes de IA para analisar tendências do mercado de ações. O sistema irá:

Coletar o ticker da ação de interesse do usuário.
Utilizar um agente para buscar notícias, análises e dados históricos relevantes sobre a ação no Google.
Utilizar outro agente para analisar as informações coletadas e identificar tendências de alta ou baixa, fatores que influenciam o preço da ação e possíveis previsões.
Apresentar a análise da tendência do mercado de ações ao usuário.

In [10]:
# Instalação de Bibliotecas
%pip -q install google-genai
!pip install -q google-adk

# Configuração da API Key
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Configuração do Cliente Gemini
from google import genai

client = genai.Client()
MODEL_ID = "gemini-2.0-flash"  # Ou outro modelo mais adequado

# Funções Auxiliares
from IPython.display import HTML, Markdown
import textwrap
from google.genai import types
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from datetime import date
import warnings

warnings.filterwarnings("ignore")

def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Definição dos Agentes
def agente_buscador_acoes():
    buscador_acoes = Agent(
        name="agente_buscador_acoes",
        model="gemini-2.0-flash",
        instruction="""
        Você é um agente de busca de informações sobre ações. Sua tarefa é utilizar a ferramenta google_search
        para buscar notícias, análises, dados históricos e outras informações relevantes sobre a ação
        fornecida pelo usuário.
        """,
        description="Agente que busca informações sobre ações no Google.",
        tools=[google_search]
    )
    return buscador_acoes

def agente_analista_acoes():
    analista_acoes = Agent(
        name="agente_analista_acoes",
        model="gemini-2.0-flash",
        instruction="""
        Você é um analista de ações. Sua tarefa é analisar as informações fornecidas,
        identificar tendências de alta ou baixa, fatores que influenciam o preço da ação
        (notícias, resultados financeiros, eventos do setor, etc.) e fornecer possíveis previsões
        sobre o comportamento futuro da ação.
        """,
        description="Agente que analisa tendências do mercado de ações.",
    )
    return analista_acoes

# Fluxo de Execução do Sistema
print("📈 Iniciando o Sistema de Análise de Tendências do Mercado de Ações 📈")

# Agentes
buscador_acoes = agente_buscador_acoes()
analista_acoes = agente_analista_acoes()

# Coletar o ticker da ação do usuário
ticker_acao = input("Digite o ticker da ação que deseja analisar (ex: AAPL, GOOGL): ")

# Buscar as informações sobre a ação
mensagem_buscador_acoes = f"Buscar informações sobre a ação: {ticker_acao}"
informacoes_acao = call_agent(buscador_acoes, mensagem_buscador_acoes)

# Analisar as informações e fornecer a análise da tendência
mensagem_analista_acoes = f"Analisar as informações da ação {ticker_acao}: {informacoes_acao}"
analise_tendencia = call_agent(analista_acoes, mensagem_analista_acoes)

# Apresentar a análise ao usuário
print("\n--- 📝 Análise da Tendência do Mercado de Ações ---\n")
display(to_markdown(analise_tendencia))

# Exemplo de Execução
# (O exemplo está no código acima)

📈 Iniciando o Sistema de Análise de Tendências do Mercado de Ações 📈
Digite o ticker da ação que deseja analisar (ex: AAPL, GOOGL): Qual o valor da ação  Microsoft BDR atualmente? Ela tem perspectiva de crescimento?

--- 📝 Análise da Tendência do Mercado de Ações ---



> ## Análise da Ação Microsoft BDR (MSFT34)
> 
> **Resumo:** A Microsoft BDR (MSFT34) apresenta um valor de mercado em torno de R$106,50, com recente valorização positiva. As perspectivas de crescimento para a ação são otimistas, impulsionadas por diversos fatores, incluindo resultados financeiros sólidos, investimentos em IA e o potencial do setor de nuvem.
> 
> **Tendências Identificadas:**
> 
> *   **Tendência de Alta:** A ação MSFT34 demonstra uma tendência de alta, corroborada pelo fechamento positivo recente e pelas previsões de analistas da Wall Street.
> *   **Crescimento Contínuo:** A Microsoft tem demonstrado crescimento contínuo, sustentado por resultados financeiros positivos e investimentos estratégicos.
> 
> **Fatores que Influenciam o Preço da Ação:**
> 
> *   **Resultados Financeiros:** O desempenho financeiro da Microsoft, especialmente o crescimento em nuvem e produtividade, exerce um impacto direto no preço da ação. Resultados positivos tendem a impulsionar o preço, enquanto resultados negativos podem levar a quedas.
> *   **Investimentos em IA:** Os investimentos da Microsoft em Inteligência Artificial (IA) são um fator crucial. A crescente demanda por serviços de IA e os consequentes investimentos da empresa sinalizam um potencial de crescimento futuro, influenciando positivamente o preço da ação.
> *   **Aquisição da Activision:** A aquisição da Activision é um evento significativo que pode afetar a composição de margem dos negócios de jogos da Microsoft a longo prazo. O impacto desta aquisição no desempenho financeiro geral da empresa influenciará o preço da ação.
> *   **Setor de Nuvem:** O potencial de crescimento do setor de armazenamento e processamento de dados em nuvem é um fator chave. O crescimento projetado de 20% para o próximo ano indica um ambiente favorável para a Microsoft, que é um dos principais players nesse mercado.
> *   **Sentimento do Mercado:** O sentimento geral do mercado em relação à tecnologia e ações de crescimento também desempenha um papel. Um mercado otimista tende a beneficiar ações como a MSFT34.
> 
> **Previsões Possíveis:**
> 
> *   **Continuação da Valorização:** Considerando os fatores mencionados, é provável que a ação MSFT34 continue a se valorizar no futuro.
> *   **Crescimento Impulsionado por IA e Nuvem:** O crescimento da Microsoft será impulsionado principalmente pelos seus investimentos em IA e pela expansão do setor de nuvem.
> *   **Volatilidade:** É importante notar que o mercado de ações está sujeito a volatilidade. Eventos inesperados, mudanças nas condições econômicas ou notícias negativas sobre a Microsoft podem impactar o preço da ação.
> 
> **Recomendações:**
> 
> *   **Monitoramento Contínuo:** É fundamental monitorar continuamente os resultados financeiros da Microsoft, os desenvolvimentos no setor de IA e nuvem, e o impacto da aquisição da Activision.
> *   **Considerar Riscos:** Antes de tomar qualquer decisão de investimento, é importante considerar os riscos associados ao mercado de ações e diversificar a carteira.
> 
> **Disclaimer:** Esta análise é baseada nas informações fornecidas e não constitui uma recomendação de investimento. É fundamental consultar um profissional financeiro antes de tomar qualquer decisão de investimento.
> 
